# Build Model Workbook

In [11]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np

Load Dataset

In [2]:
data_dir = 'data'
processed_data = 'processed_data'

In [3]:
dataset = pd.read_csv(os.path.join(processed_data, 'dataset.csv'))

In [4]:
dataset.head(10)

,center_image,left_image,right_image,steering_angle,throttle,break,speed,folder
0,center_2019_05_24_13_42_51_893.jpg,left_2019_05_24_13_42_51_893.jpg,right_2019_05_24_13_42_51_893.jpg,0.0,0.000000,0.0,24.62923,driving_data_2
1,center_2019_05_24_13_42_51_970.jpg,left_2019_05_24_13_42_51_970.jpg,right_2019_05_24_13_42_51_970.jpg,0.0,0.000000,0.0,24.43121,driving_data_2
2,center_2019_05_24_13_42_52_041.jpg,left_2019_05_24_13_42_52_041.jpg,right_2019_05_24_13_42_52_041.jpg,0.0,0.000000,0.0,24.23477,driving_data_2
3,center_2019_05_24_13_42_52_111.jpg,left_2019_05_24_13_42_52_111.jpg,right_2019_05_24_13_42_52_111.jpg,0.0,0.000000,0.0,24.08849,driving_data_2
4,center_2019_05_24_13_42_52_179.jpg,left_2019_05_24_13_42_52_179.jpg,right_2019_05_24_13_42_52_179.jpg,0.0,0.000000,0.0,23.94309,driving_data_2
5,center_2019_05_24_13_42_52_246.jpg,left_2019_05_24_13_42_52_246.jpg,right_2019_05_24_13_42_52_246.jpg,0.0,0.000000,0.0,23.75058,driving_data_2
6,center_2019_05_24_13_42_52_316.jpg,left_2019_05_24_13_42_52_316.jpg,right_2019_05_24_13_42_52_316.jpg,0.0,0.000000,0.0,23.60721,driving_data_2
7,center_2019_05_24_13_42_52_384.jpg,left_2019_05_24_13_42_52_384.jpg,right_2019_05_24_13_42_52_384.jpg,0.0,0.118662,0.0,23.44914,driving_data_2
8,center_2019_05_24_13_42_52_451.jpg,left_2019_05_24_13_42_52_451.jpg,right_2019_05_24_13_42_52_451.jpg,0.0,0.318925,0.0,23.45194,driving_data_2
9,center_2019_05_24_13_42_52_526.jpg,left_2019_05_24_13_42_52_526.jpg,right_2019_05_24_13_42_52_526.jpg,0.0,0.547578,0.0,23.67355,driving_data_2


# Build Simple Dataset

center image only with steering angles

In [5]:
x_set = []
y_set = []
for rn, row in dataset.iterrows():
    path = os.path.join('.', data_dir, row['folder'], 'IMG', row['center_image'])
    image = plt.imread(path)
    x_set.append(image)
    y_set.append(row['steering_angle'])

In [6]:
len(x_set)

7921

In [7]:
dataset['folder'].unique()

array(['driving_data_2', 'driving_data_1', 'driving_data_3'], dtype=object)

In [9]:
x_set[0].shape

(160, 320, 3)

In [12]:
x_train = np.array(x_set)
y_train = np.array(y_set)

In [13]:
x_train.shape

(7921, 160, 320, 3)

In [14]:
y_train.shape

(7921,)

# Simple Model

In [8]:
from keras.models import Sequential
from keras.layers import Flatten, Dense

Using TensorFlow backend.


In [15]:
model = Sequential()
model.add(Flatten(input_shape=(160, 320, 3)))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
model.fit(x_train, y_train)

Epoch 1/10
7921/7921 [==============================] - 2s 273us/step - loss: 6330750.7567
Epoch 2/10
7921/7921 [==============================] - 2s 268us/step - loss: 3189.8016
Epoch 3/10
7921/7921 [==============================] - 2s 266us/step - loss: 2260.3092
Epoch 4/10
7921/7921 [==============================] - 2s 269us/step - loss: 3153.2086
Epoch 5/10
7921/7921 [==============================] - 2s 267us/step - loss: 2191.0430
Epoch 6/10
7921/7921 [==============================] - 2s 268us/step - loss: 2950.0167
Epoch 7/10
7921/7921 [==============================] - 2s 264us/step - loss: 4504.1209
Epoch 8/10
7921/7921 [==============================] - 2s 267us/step - loss: 24501.2849
Epoch 9/10
7921/7921 [==============================] - 2s 267us/step - loss: 1336917.8388
Epoch 10/10
7921/7921 [==============================] - 2s 263us/step - loss: 90715.8200
